In [25]:
import os
ON_KAGGLE = os.path.exists('/kaggle/input')
ENABLE_SUBMISSION = False
os.environ["ON_KAGGLE"] = str(ON_KAGGLE)

In [22]:
# Install required packages if on Kaggle
if ON_KAGGLE:
    !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/parglare-0-18-0/parglare-0.18.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/rank-bm25-whl/rank_bm25-0.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/amino-gaz-whl/aimo_gaz-0.0.1-py3-none-any.whl -qq --no-deps --force-reinstall
else:
    !pip install -r requirements.txt
    !pip install --upgrade build
    !python -m build
    !pip install dist/aimo_gaz-0.0.1-py3-none-any.whl --force-reinstall

In [23]:
from aimo_gaz.models.llm_helper import hello_world

hello_world()

PyTorch version: 2.1.2
Hello World!


In [26]:
# Set up the evaluation API
if ENABLE_SUBMISSION and ON_KAGGLE:
    import aimo

    env = aimo.make_env()
    iter_test = env.iter_test()

    # Iterate through the test set and use the model make predictions
    for test, sample_submission in iter_test:
        sample_submission['answer'] = 100 # TODO: call the model to predict the solution for text['problem']
        env.predict(sample_submission)
        print(test)
        print(sample_submission, '\n')